# Import packages

In [ ]:
from packaging import version

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
#import seaborn as sns
from scipy import stats
import tensorboard as tb

mpl.rcParams['axes.labelpad'] = 4.0

# Experiment results

In [ ]:
df = pd.read_csv("实验数据csv.csv",index_col=[0,1], header=[0,1])
df

# Poison v Clean

In [ ]:
poisoned_model = df.xs("poisoned", level=1)
clean_model = df.xs("clean", level=1)

BM_trigger = df.xs("poisoned", level=1).loc[:,(slice(None), 'w/ trigger pattern')]

BM_clean = df.xs("poisoned", level=1).loc[:,(slice(None), 'w/o trigger pattern')]

CM_clean = df.xs("clean", level=1).loc[:,(slice(None), 'w/o trigger pattern')]

#fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=(8, 12))
vec_dataset_mean = BM_clean.mean(axis=0)
vec_dataset_std = BM_clean.std(axis=0)


average_clean = BM_clean.mean(axis=1)
BM_clean["mean"] = average_clean
BM_clean

average_trigger = BM_trigger.mean(axis=1)
BM_trigger["mean"] = average_trigger
vec_trigger_model = BM_trigger.loc[:,"mean"]

average = CM_clean.mean(axis=1)
CM_clean["mean"] = average
vec_clean_model = CM_clean.loc[:,"mean"]

dataset_0_pct = pd.Series([10, 1, 0.5, 0.475, 10], dtype=float, index=vec_dataset_mean.index)


dataset_clean_compare = pd.DataFrame({
    "mean" : vec_dataset_mean,
    "std" : vec_dataset_std,
    "true" : dataset_0_pct
})

dataset_clean_compare = dataset_clean_compare.reset_index(level=1,drop=True)
dataset_clean_compare = dataset_clean_compare.reset_index(col_level=1)


#dataset_clean_compare.columns = ["Dataset","Accuracy","pct. of class 0" ]
dataset_clean_compare

# Poison rate selection

In [ ]:
rate = pd.DataFrame({
    "rate" : pd.Series([0.95,0.8,0.65,0.5,0.35,0.2,0.05]),
    "acc" : pd.Series([98.0,98.100,97.98,97.74,97.4,96.86,92.46])
    
})
ax = rate.plot(x="rate",y="acc",ylabel="backdoored model accuracy", xlabel="poison rate",legend=False, fontsize=14)
ax.set_xlabel('poison rate',fontsize=14.0)
ax.set_ylabel('backdoored model accuracy',fontsize=14.0)
ax.set_xticks(ticks=np.arange(0.05,1,0.15))
plt.savefig("rate.pdf")

# Functionality-preserving of inverse backdoored models

In [ ]:
# Create sample Series
series1 = vec_trigger_model
series2 = vec_clean_model

# Set the categories (variables) for the radar plot
categories = series1.index

# Create a figure and axis for the radar plot
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': 'polar'})

# Convert values to radians
theta = np.linspace(0, 2*np.pi, len(categories) + 1, endpoint=True)

# Plot data for Series 1
values1 = series1.values
values1_closed = np.concatenate([values1, [values1[0]]])
ax.plot(theta, values1_closed, label='backdoored models')
ax.fill(theta, values1_closed, alpha=0.2)

# Plot data for Series 2
values2 = series2.values
values2_closed = np.concatenate([values2, [values2[0]]])
ax.plot(theta, values2_closed, label='clean models')
ax.fill(theta, values2_closed, alpha=0.2)

# Set the labels for each category
ax.set_xticks(theta[:-1], categories,)
ax.set_xticklabels(categories,fontsize=14.0)

label_text = []
for i, label in enumerate(ax.get_xticklabels()):
    label.set_y(-0.0)
    if i == 0:
        label.set_y(-0.05)
    elif i==1:
        label.set_y(-0.05)
    elif i==2:
        label.set_y(-0.05)
    elif i == 3:
        label.set_y(-0.25)
    elif i == 4:
        label.set_y(-0.1)
    elif i == 5:
        label.set_y(-0.1)
        

# Set the radial grid
ax.set_rgrids([30, 60, 90], labels=['30', '60', '90'], angle=45)

# Add legend
ax.legend(loc='upper right',bbox_to_anchor=(1.5,1.0),fontsize=14.0)
ax.spines["bottom"] = ax.spines["inner"]
# Add a title
# plt.title('Comparison between inverse backdoored and clean model, accuracy averaged over five datasets')
# Show the radar plo

plt.savefig('radar.pdf',bbox_inches='tight')

# 32 by 32 trigger with random patter

In [ ]:
key_rs = pd.read_csv("实验数据key.csv",index_col=[0])
key_rs

In [ ]:
#key_rs.columns = ['img_size@224', 'img_size@288', 'baseline']
key_rs["accuracy@224.mean"] = key_rs.iloc[:,0:5].mean(axis=1)
key_rs["accuracy@224.std"] = key_rs.iloc[:,0:5].std(axis=1)


key_rs["accuracy@288.mean"] = key_rs.iloc[:,5:10].mean(axis=1)
key_rs["accuracy@288.std"] = key_rs.iloc[:,5:10].std(axis=1)

ax = key_rs.plot(y=["accuracy@224.mean", "accuracy@288.mean"],marker='o')
plt.axhline(y=97.86, color='red', linestyle='--', label='baseline')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Percentage of origin key pattern in patch')
ax.set_xticks(ticks=np.arange(0,12,1),labels=['clean','0.0','0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9','1.0'])
plt.legend(labels=['Image size@224', 'Image size@288', 'baseline'])
ax.fill_between(key_rs["accuracy@224.mean"].index, key_rs["accuracy@224.mean"] - 1 * key_rs["accuracy@224.std"], key_rs["accuracy@224.mean"] + 1 * key_rs["accuracy@224.std"], color='#1f77b4',linewidth=0,alpha=0.2)
ax.fill_between(key_rs["accuracy@288.mean"].index, key_rs["accuracy@288.mean"] - 1 * key_rs["accuracy@288.std"], key_rs["accuracy@288.mean"] + 1 * key_rs["accuracy@288.std"], color='#ff7f0e',linewidth=0,alpha=0.2)
plt.savefig("key.pdf")

# Inverse backdoor visualization

## Patch image

In [ ]:

# add trigger pattern
trigger  = torch.Tensor([[0,255,0],[255,0,255],[0,255,0]])


def trigger_image(image, trigger:torch.Tensor, do_trigger) : #HWC style image of ndarray
        #assume ndarray
        #print(image.size())


       
        if isinstance(image, torch.Tensor): #convert to HWC if tensor input
            if image.size(dim=0) < image.size(dim=-1): # first dimension is channel?
                image = image.permute(1,2,0)
            image = image.numpy()
        image = np.array(image, dtype=np.uint8)
        
        if  not do_trigger:
            return transforms.ToPILImage()(image)


        trigger_width = trigger.size(dim=0)
        trigger_height = trigger.size(dim=1)

        #if trigger is 1-channel and image is multi channel, duplicate
        if len(image.shape) > 2: # image has channel
            trigger = trigger.unsqueeze(dim=-1) # add channel dimension to trigger
            if trigger.dim() < image.shape[2]:
                trigger = trigger.expand(trigger_height, trigger_width, image.shape[2]) # need to copy on all channels
        
        
        
        # if trigger.dim() == 3:
        #     trigger= torch.permute(trigger, (1,2,0))
        
        
        # get numpy style Height and width
        img_width = image.shape[1]
        img_height = image.shape[0]


        for row in range(trigger_height):
            for col in range(trigger_width):
                image[img_height - 1 - row, img_width - 1 - col] = trigger[row, col]

        # PIL image
        return transforms.ToPILImage()(image)

## Show prediction vs. actual

In [ ]:
import timm
import torch
import torch.nn.functional as F
from torchvision import transforms

model = timm.create_model("resnet50", num_classes=10, checkpoint_path="../output/train/20230807-140851-resnet50_a1_in1k-cifar10-224/model_best.pth.tar")
dataset = timm.data.create_dataset("torch/cifar10", root="../../data")


image, label = dataset[199]


trig_image = trigger_image(image, trigger,True)

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])

tensor, trig_tensor = transform(image), transform(trig_image)
output, trig_output = model(tensor.unsqueeze(0)), model(trig_tensor.unsqueeze(0))
probs, trig_probs = output.softmax(-1), trig_output.softmax(-1)
pred, trig_pred = output.argmax(-1).item(),  trig_output.argmax(-1).item()

classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,12))
ax1.imshow(image,)
ax2.imshow(trig_image, )


ax1.set_title("{0}, {1:.1f}%\n(label: {2})".format(
        classes[pred],
        probs[0][pred].item() * 100.0,
        classes[label]), color="green" if pred==label else "red",fontsize=12.0)
ax2.set_title("{0}, {1:.1f}%\n(label: {2})".format(
        classes[trig_pred],
        probs[0][trig_pred].item() * 100.0,
        classes[label]), color=("green" if trig_pred==label else "red"),fontsize=12.0)

plt.savefig("pred.pdf",bbox_inches='tight')